In [1]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.9.0, build 1664956232
Copyright (C) 2014-2025 Monash University, NICTA, Data61


In [2]:
import numpy as np
from math import ceil
import random

In [3]:
num_staff = 10 # number of staff members
num_days = 5 # number of days

# leave_capacity = [ceil((random.randrange(0, 50, 5) / 100) * num_staff) for i in range(num_days)] # random capacity for each day
leave_capacity = [1,1,1,1,1] # random capacity for each day

print(f"Capacity Matrix: \n {leave_capacity}")

num_ones = int(num_staff * num_days * 0.4)
num_zeros = (num_staff * num_days) - num_ones
array = np.array([1] * num_ones + [0] * num_zeros)
np.random.shuffle(array)
preference_matrix = array.reshape(num_staff,num_days)

print(f"Preference Matrix: \n {preference_matrix}")


preference_matrix = preference_matrix.tolist()



Capacity Matrix: 
 [1, 1, 1, 1, 1]
Preference Matrix: 
 [[0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 1]
 [0 0 1 0 1]
 [1 0 0 1 0]
 [0 0 1 1 1]
 [1 0 1 0 0]
 [0 1 0 1 1]
 [1 0 0 0 1]
 [0 0 1 1 0]]


In [4]:
%%minizinc

include "globals.mzn";

% Existing Variables
int: num_staff;
int: num_days;
array[1..num_days] of int: leave_capacity;
array[1..num_staff, 1..num_days] of int: preference_matrix;

% Matrix to fill in
array[1..num_staff, 1..num_days] of var 0..1: leave;

constraint
  forall(d in 1..num_days)(
    sum(p in 1..num_staff)(leave[p,d]) <= leave_capacity[d]
  );

var int: satisfaction = sum(x in 1..num_staff, y in 1..num_days)(preference_matrix[x, y] * leave[x, y]);

solve maximize satisfaction;

{'leave': [[0, 1, 0, 0, 0],
  [0, 0, 0, 0, 1],
  [0, 0, 0, 1, 0],
  [0, 0, 1, 0, 0],
  [1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]]}